In [ ]:
import ixmp as ix
import message_ix
import pandas as pd

mp = ix.Platform("ixmp_dev")
scenario = message_ix.Scenario(mp, model="MESSAGEix-Materials", scenario="NoPolicy_petro_thesis_2")#, version=37)

## Generate new mode for methanol technologies

In [ ]:
par_dict = {}
for i in scenario.par_list():
    try: par_dict[i] = scenario.par(i, filters={"technology":["meth_ng", "meth_h2", "meth_coal", "meth_bio", "meth_bio_ccs", "meth_coal_ccs", "meth_ng_ccs"]})
    except:
        pass

In [ ]:
par_dict = {}
for i in scenario.par_list():
    try:
        df = scenario.par(i, filters={"technology":"meth_bal"})
        if df.size != 0:
            par_dict[i] = df
    except:
        pass

In [ ]:
for i in par_dict.keys():
    if "mode" in par_dict[i].columns:
        par_dict[i]["mode"] = "feedstock"

In [ ]:
keys = list(par_dict.keys())
for i in keys:
    if par_dict[i].size == 0:
        par_dict.pop(i)

In [ ]:
for j in par_dict.keys():
    df = par_dict[j]
    if "mode" in par_dict[j].columns:
        df["mode"] = "M2"

df = par_dict["output"]
df.loc[df["commodity"]=="methanol","level"] = "final_material"
par_dict["output"] = df

In [ ]:
#scenario.remove_solution()
#scenario.check_out()
for i in par_dict.keys():
    print(i)
    scenario.check_out()
    scenario.add_par(i, par_dict[i])
    scenario.commit("add new mode for meth tecs")

In [ ]:
scenario.par("input", filters={"technology":"meth_coal", "mode":"M2"})

## update meth_bio(ccs)

In [ ]:
#df = scenario.par("output", filters={"technology":"meth_bio_ccs"})
#scenario.remove_solution()
scenario.check_out()
scenario.add_par("output", df[df["year_act"] > 2025])
#df = scenario.par("input", filters={"technology":"meth_bio_ccs"})
#scenario.add_par("input", df[df["year_act"] > 2025])
scenario.commit(comment="fix meth_bio_ccs issue")

In [ ]:
scenario.check_out()
df = scenario.par("input", filters={"technology":"meth_bio_ccs"})
scenario.remove_par("input", df)
scenario.add_par("input", df[df["year_act"] > 2025])

In [ ]:
scenario.commit("fix meth_bio_ccs input")

In [ ]:
df1 = scenario.par("input", filters={"technology":"meth_bio_ccs"})
df1

## add missing years to methanol trade tecs

In [ ]:
paramList_tec = [x for x in scenario.par_list() if "technology" in scenario.idx_sets(x)]

In [ ]:
par_dict_exp = {}
df_all_rows = pd.DataFrame()
for p in paramList_tec:
    df = scenario.par(p, filters={"technology":"eth_bio_ccs"})
    if df.index.size:
        par_dict_exp[p] = df

list(par_dict_exp.keys())

In [ ]:
par_dict_exp.keys()

In [ ]:
par_dict_exp["relation_activity"]

In [ ]:
for p in par_dict_exp.keys():
    print(p, len(par_dict_exp[p].value.unique()))
    try:
        print(par_dict_exp[p]["year_act"].min())
        print(par_dict_exp[p]["year_vtg"].min())
        if par_dict_exp[p]["year_act"].min() < 2020:
            print(par_dict_exp[p])
    except:
        continue

In [ ]:
import numpy as np
par_dict_bc = {}
for p in list(par_dict_exp.keys())[:-2]:
    df = par_dict_exp[p].copy(deep=True)
    if "year_act" in df.columns:
        year_name = "year_act"
    else:
        year_name = "year_vtg"
    min_year = df[year_name].min()
    min_year = 2040
    new_lowest_y = 2000
    tec_lt = 20
    new_years = np.linspace(new_lowest_y, min_year, int((min_year-new_lowest_y)/5)+1).astype(int)[:-1]
    df_min_year = df[df[year_name] == min_year].copy(deep=True)
    print(p, new_years)
    if len(new_years) == 0:
        continue
    df_min_year_new = df_min_year.copy(deep=True)
    for i in new_years:
        df_min_year_add = df_min_year.copy(deep=True)
        df_min_year_add[year_name] = i
        if (year_name == "year_act") & ("year_vtg" in df.columns):
            df_min_year_add = df_min_year_add[df_min_year_add["year_vtg"] == min_year]
            df_min_year_add["year_vtg"] = i
            old_years = np.linspace(i-tec_lt, i, int((tec_lt)/5)+1).astype(int)
            print(i, old_years)
            for j in old_years:
                df_min_year_add_vtg = df_min_year_add[df_min_year_add["year_vtg"]==df_min_year_add["year_act"]].copy(deep=True)
                df_min_year_add_vtg["year_vtg"] = j
                df_min_year_add = pd.concat([df_min_year_add, df_min_year_add_vtg])
        df_min_year_new = pd.concat([df_min_year_new, df_min_year_add])
    par_dict_bc[p] = df_min_year_new

In [ ]:
for i in par_dict_bc.keys():
    par_dict_bc[i] = par_dict_bc[i].drop_duplicates()
    if "year_act" in par_dict_bc[i].columns:
        par_dict_bc[i] = par_dict_bc[i][par_dict_bc[i]["year_act"] >= 2020]
    if "year_vtg" in par_dict_bc[i].columns:
        par_dict_bc[i] = par_dict_bc[i][par_dict_bc[i]["year_vtg"] < 2020]


keys = list(par_dict_bc.keys())
for i in keys:
    if "year_vtg" not in par_dict_bc[i].columns:
        par_dict_bc.pop(i)

In [ ]:
df = par_dict_bc["input"]
df[df["year_act"]==2035]

In [ ]:
par_dict_imp = {}
df_all_rows = pd.DataFrame()
for p in paramList_tec:
    df = scenario.par(p, filters={"technology":"meth_imp"})
    if df.index.size:
        par_dict_imp[p] = df
par_dict_imp.keys()

In [ ]:
par_dict_bc_imp = {}
for p in list(par_dict_imp.keys())[:-2]:
    df = par_dict_imp[p].copy(deep=True)
    if "year_act" in df.columns:
        year_name = "year_act"
    else:
        year_name = "year_vtg"
    min_year = df[year_name].min()
    min_year = 2045
    print(p)
    new_years = np.linspace(2020, min_year, int((min_year-2020)/5)+1).astype(int)[:-1]
    df_min_year = df[df[year_name] == min_year].copy(deep=True)
    if len(new_years) == 0:
        continue
    for i in new_years:
        df_min_year_add = df_min_year.copy(deep=True)
        df_min_year_add[year_name] = i
        print(new_years)
        if (year_name == "year_act") & ("year_vtg" in df.columns):
            df_min_year_add["year_vtg"] = i
        if (year_name == "year_act") & ("year_rel" in df.columns):
            df_min_year_add["year_rel"] = i
        df_min_year = pd.concat([df_min_year, df_min_year_add])
    par_dict_bc_imp[p] = df_min_year

In [ ]:
for i in par_dict_bc_imp.keys():
    par_dict_bc_imp[i] = par_dict_bc_imp[i].drop_duplicates()

In [ ]:
par_dict_trd = {}
df_all_rows = pd.DataFrame()
for p in paramList_tec:
    df = scenario.par(p, filters={"technology":"meth_trd"})
    if df.index.size:
        par_dict_trd[p] = df
par_dict_trd.keys()

In [ ]:
par_dict_bc_trd = {}
for p in list(par_dict_trd.keys())[:-1]:
    df = par_dict_trd[p].copy(deep=True)
    if "year_act" in df.columns:
        year_name = "year_act"
    else:
        year_name = "year_vtg"
    min_year = df[year_name].min()
    min_year = 2045
    print(p)
    new_years = np.linspace(2020, min_year, int((min_year-2020)/5)+1).astype(int)[:-1]
    df_min_year = df[df[year_name] == min_year].copy(deep=True)
    if len(new_years) == 0:
        continue
    for i in new_years:
        df_min_year_add = df_min_year.copy(deep=True)
        df_min_year_add[year_name] = i
        print(new_years)
        if (year_name == "year_act") & ("year_vtg" in df.columns):
            df_min_year_add["year_vtg"] = i
        if (year_name == "year_act") & ("year_rel" in df.columns):
            df_min_year_add["year_rel"] = i
        df_min_year = pd.concat([df_min_year, df_min_year_add])
    par_dict_bc_trd[p] = df_min_year

In [ ]:
for i in par_dict_bc_trd.keys():
    par_dict_bc_trd[i] = par_dict_bc_trd[i].drop_duplicates()

In [ ]:
list(par_dict_exp.keys())[7:-3]

In [ ]:
#par_dict_bc_imp.keys()
#par_dict_bc_trd.keys()
#par_dict_bc.keys()

with pd.ExcelWriter('meth_bio_ccs_additions.xlsx') as writer:
    for i in list(par_dict_exp.keys())[7:-3]:
        par_dict_exp[i].to_excel(writer, sheet_name=i, index=False)

In [ ]:
scenario.commit("update meth costs")

In [ ]:
df_bio = scenario.par("inv_cost", filters={"technology": "meth_coal"})
df_bio_ccs = scenario.par("inv_cost", filters={"technology": "meth_coal_ccs"})
df_bio.set_index(["node_loc", "year_vtg"])["value"]-df_bio_ccs.set_index(["node_loc", "year_vtg"])["value"]

In [ ]:
df = scenario.par("land_input")

In [ ]:
df.groupby(["land_scenario", "year"]).sum().reset_index().set_index("year").loc[2050].sort_values("value")#.plot(x="year")

In [ ]:
scenario.par("output", filters={"technology":""})

In [ ]:
from message_ix_models.util import broadcast, same_node
bio_dict = data_methanol.gen_data_meth_bio(scenario)

df_all = bio_dict["input"]
yv_ya = scenario.vintage_and_active_years()
yv_ya = yv_ya[(yv_ya["year_act"]>2015) & (yv_ya["year_act"]-yv_ya["year_vtg"]<20)]
nodes=df_all["node_loc"].unique()

import pandas as pd

df=df_all
df_new = message_ix.make_df("input", **yv_ya, **df.drop(["year_vtg", "year_act", "node_loc"], axis=1).iloc[0]).pipe(broadcast, node_loc=nodes).pipe(same_node)
df_new.loc[df_new["year_vtg"]<2030, "value"] = 1.7241
df_new.loc[df_new["year_vtg"].isin([2030, 2035]), "value"] = 1.6393
df_new.loc[df_new["year_vtg"].isin([2040, 2045]), "value"] = 1.5873
df_new.loc[df_new["year_vtg"]>2045, "value"] = 1.5385

df_input_new = df_new.copy(deep=True)
df_input_new

In [ ]:
df_all = bio_dict["output"]
df_output_new = pd.DataFrame()

df = df_all[df_all["commodity"]=="d_heat"]
df_new = message_ix.make_df("output", **yv_ya, **df.drop(["year_vtg", "year_act", "node_loc"], axis=1).iloc[0]).pipe(broadcast, node_loc=nodes).pipe(same_node)
df_new.loc[df_new["year_vtg"]<2030, "value"] = 0.3793
df_new.loc[df_new["year_vtg"].isin([2030, 2035]), "value"] = 0.3607
df_new.loc[df_new["year_vtg"].isin([2040, 2045]), "value"] = 0.3492
df_new.loc[df_new["year_vtg"]>2045, "value"] = 0.3385
df_output_new = pd.concat([df_output_new, df_new])

df = df_all[df_all["commodity"]=="electr"]
df_new = message_ix.make_df("output", **yv_ya, **df.drop(["year_vtg", "year_act", "node_loc"], axis=1).iloc[0]).pipe(broadcast, node_loc=nodes).pipe(same_node)
df_new.loc[df_new["year_vtg"]<2030, "value"] = 0.03448
df_new.loc[df_new["year_vtg"].isin([2030, 2035]), "value"] = 0.0328
df_new.loc[df_new["year_vtg"].isin([2040, 2045]), "value"] = 0.03175
df_new.loc[df_new["year_vtg"]>2045, "value"] = 0.03077
df_output_new = pd.concat([df_output_new, df_new])

df = df_all[df_all["commodity"]=="methanol"]
df_new = message_ix.make_df("output", **yv_ya, **df.drop(["year_vtg", "year_act", "node_loc"], axis=1).iloc[0]).pipe(broadcast, node_loc=nodes).pipe(same_node)
df_output_new = pd.concat([df_output_new, df_new])

In [ ]:
df_all = bio_dict["capacity_factor"]
df = df_all
df_cap = message_ix.make_df("capacity_factor", **yv_ya,
                            **df.drop(["year_vtg", "year_act", "node_loc"],
                            axis=1).iloc[0]).pipe(broadcast, node_loc=nodes).pipe(same_node)
df_cap.loc[df_new["year_vtg"]<2030, "value"] = 0.9215


df_all = bio_dict["var_cost"]
df = df_all
df_var = message_ix.make_df("var_cost", **yv_ya,
                            **df.drop(["year_vtg", "year_act", "node_loc"]
                                      , axis=1).iloc[0]).pipe(broadcast, node_loc=nodes).pipe(same_node)
df_var.loc[df_new["year_vtg"]<2030, "value"] = 18.89

In [ ]:
df_all = bio_dict["fix_cost"]
df = df_all

df_fix = message_ix.make_df("fix_cost", **yv_ya, **df.drop(["year_vtg", "year_act", "node_loc"], axis=1).iloc[0]).pipe(broadcast, node_loc=nodes).pipe(same_node)
df_fix.loc[df_fix["year_vtg"]<2030, "value"] = 54.101
df_fix.loc[df_fix["year_vtg"].isin([2030, 2035]), "value"] = 36.067
df_fix.loc[df_fix["year_vtg"].isin([2040, 2045]), "value"] = 35.521
df_fix.loc[df_fix["year_vtg"]>2045, "value"] = 36.067

df_fix

In [ ]:
new_par_dict = {
    "capacity_factor":df_cap,
    "input":df_input_new,
    "output":df_output_new,
    "fix_cost":df_fix,
    "var_cost":df_var,
    "inv_cost":bio_dict["inv_cost"],
    "technical_lifetime":bio_dict["technical_lifetime"]
}
with pd.ExcelWriter('meth_bio_techno_economic.xlsx') as writer:
    for i in new_par_dict.keys():
        new_par_dict[i].to_excel(writer, sheet_name=i, index=False)

In [ ]:
#data_methanol.gen_data_methanol(scenario)["historical_new_capacity"]
#data_methanol.add_methanol_trp_additives(scenario)
#data_methanol.get_cost_ratio_2020(scenario, "meth_coal", "inv_cost", year=2020, ref_reg="R12_WEU")
df_bio = data_methanol.gen_data_meth_bio(scenario)["inv_cost"]
df_bio_ccs = data_methanol.gen_meth_bio_ccs(scenario)["inv_cost"]
df_bio.set_index(["node_loc", "year_vtg"])["value"]-df_bio_ccs.set_index(["node_loc", "year_vtg"])["value"]

In [ ]:
meth_dict = data_methanol.gen_data_methanol(scenario)

In [ ]:
meth_dict["relation_activity"]["technology"].unique()

In [ ]:

arr = ["meth_ng", "meth_coal", "meth_coal_ccs", "meth_ng_ccs"]
df = scenario.par("relation_activity", filters={"relation": "CO2_cc", "technology":arr})
df = df.rename({"year_rel":"year_vtg"}, axis=1)
values = dict(zip(df["technology"], df["value"]))

df_em = scenario.par("emission_factor", filters={"emission": "CO2_transformation", "technology":arr})
for i in arr:
    df_em.loc[df_em["technology"]==i,"value"] = values[i]
df_em["emission"] = "CO2_industry"
df_em
#scenario.add_par(df_em)

In [ ]:
df = scenario.par("input", filters={"technology":"meth_t_d"})
df["value"] = 1
df

In [ ]:
df = scenario.par("technical_lifetime")
dict = df.loc[df["technology"].str.contains("NH3"),["technology","value"]].set_index("technology").to_dict()["value"]
for i in df.keys():
    if ("year_vtg" in df[i].columns) & ("year_act" in df[i].columns):
        df_temp = df[i]
        df[i] = df_temp[(df_temp["year_act"]-df_temp["year_vtg"])<=dict[df_temp["technology"]]]

## modify meth exp limit

In [ ]:
df = scenario.par("relation_activity", filters={"relation":"meth_exp_limit"})
df_mea = df[(df["technology"]=="meth_exp") & (df["node_rel"]=="R12_MEA")]
df_mea[df_mea["value"]] = -1.15
df_lam = df[(df["technology"]=="meth_exp") & (df["node_rel"]=="R12_LAM")]
df_lam[df_lam["value"]] = -1.6
df_rel = pd.concat([df_mea, df_lam])
scenario.check_out()
scenario.add_par("relation_activity", df_rel)
scenario.commit("modify meth exp share constraint for LAM and MEA")

In [ ]:
#scenario.par("relation_lower", filters={"relation":"meth_exp_limit"})
rel_dict = {
    "relation": "meth_exp_tot",
    "year_rel": 2020,
    "year_act": 2020,
    "technology": "meth_exp_tot",
    "mode": ["fuel", "feedstock"],
    "unit": "-",
    "value": 1

}
from message_ix_models.util import broadcast, same_node
nodes = scenario.set("node")[1:]
nodes_share = nodes.drop(5).reset_index(drop=True)
df = message_ix.make_df("relation_activity", **rel_dict).pipe(broadcast, node_loc=nodes_share)
df["node_rel"] = df["node_loc"]
df[["relation", "node_rel", "year_rel", "node_loc", "technology", "year_act", "mode", "value", "unit"]]
# rel_dict = {
#     "relation": "meth_exp_tot",
#     "year_rel": 2020,
#     "year_act": 2020,
#     "technology": "meth_exp_tot",
#     "mode": ["fuel", "feedstock"],
#     "unit": "-",
# }
# df_up = message_ix.make_df("relation_upper", **rel_dict).pipe(broadcast, node_rel=nodes_share)
# df_lo = message_ix.make_df("relation_lower", **rel_dict).pipe(broadcast, node_rel=nodes_share)

In [ ]:
df = scenario.par("relation_lower", filters={"relation":"meth_exp_tot"})
scenario.check_out()
scenario.remove_par("relation_lower", df)


In [ ]:
scenario.commit("remove meth exp tot upper")